<a href="https://colab.research.google.com/github/KimDukJung/bbc/blob/main/movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1번 블록

#사용할 패키지 임포트

from time import time  # 경과 시간 확인
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences  # 데이터를 패딩(채우기)
from keras.models import Sequential # 순차적으로 층을 쌓겠다. 입력에서 출력방향으로
from keras.layers import Embedding, LSTM, Dense

# 하이퍼 파라미터 설정
MY_LENGTH = 80  # 영화평 단어 수
MY_WORD = 5000  # 사전 단어 수
MY_SAMPLE = 5   # 샘플 영화평


In [ ]:
# 2번 블록

# keras로 데이터 파일 준비
(X_train, Y_train), (X_test, Y_test)=imdb.load_data(num_words=MY_WORD)

# 샘플 영화평 출력
print('샘플 영화평:\n', X_train[MY_SAMPLE])
print('총 단어 수:\n', len(X_train[MY_SAMPLE]))
print('감성 (0=부정, 1=긍정):', Y_train[MY_SAMPLE])

샘플 영화평:
 [1, 778, 128, 74, 12, 630, 163, 15, 4, 1766, 2, 1051, 2, 32, 85, 156, 45, 40, 148, 139, 121, 664, 665, 10, 10, 1361, 173, 4, 749, 2, 16, 3804, 8, 4, 226, 65, 12, 43, 127, 24, 2, 10, 10]
총 단어 수:
 43
감성 (0=부정, 1=긍정): 0


In [ ]:
# 3번 블록

# 각 영화평의 길이를 일정하게 맞춤
X_train = pad_sequences(sequences=X_train,  # 패딩할 원본 데이터
                        truncating = 'post',   # 영화평 길이가 우리가 지정한 80 단어보다 길면 뒤쪽(post)을 자르겠다.
                        padding = 'post',   # 영화평 길이가 80단어보다 짧으면 뒤쪽에 0을 패딩하겠다.
                        maxlen=MY_LENGTH)   # 최대 길이는 MY_LENGTH로 지정한 80단어로 하겠다.
X_test = pad_sequences(sequences=X_test,
                       truncating = 'post',
                       padding = 'post',
                       maxlen=MY_LENGTH)

# 최종 데이터 모양 출력
print('\n학습용 입력 데이터 모양 : ', X_train.shape)
print('학습용 출력 데이터 모양 : ', Y_train.shape)
print('테스트용 입력 데이터 모양 : ', X_test.shape)
print('테스트용 출력 데이터 모양 : ', Y_test.shape)


학습용 입력 데이터 모양 :  (25000, 80)
학습용 출력 데이터 모양 :  (25000,)
테스트용 입력 데이터 모양 :  (25000, 80)
테스트용 출력 데이터 모양 :  (25000,)


In [ ]:
# 4번 블록

# RNN 구현
model = Sequential()     # RNN 모델의 층을 순차적으로(입력 -> 출력) 쌓겠다.
model.add(Embedding(input_dim=MY_WORD, output_dim=32, input_length=MY_LENGTH))
model.add(LSTM(units=64 , input_shape=(MY_LENGTH , 32)))
model.add(Dense(units=1, activation='sigmoid'))

print('RNN 요약')
model.summary()

RNN 요약


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 5번 블록

# RNN 학습
model.compile(optimizer='adam',    # 최적화 함수
              loss='binary_crossentropy',   # 손실 함수
              metrics=['acc'])     # 매 에포크마다 출력할 값 ( acc = 정확도 )
start = time()

model.fit(x = X_train,  # 학습용 입력 데이터 설정
          y = Y_train,   # X_train에 대응되는 정답 데이터
          epochs = 10,
          batch_size = 200,
          verbose = 1  )

end = time()
print('\n 총 학습 시간 : {:.2f}초'.format(end - start))

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 152ms/step - acc: 0.5852 - loss: 0.6441
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - acc: 0.8290 - loss: 0.3962
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 162ms/step - acc: 0.8521 - loss: 0.3463
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 148ms/step - acc: 0.8720 - loss: 0.3102
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 160ms/step - acc: 0.8791 - loss: 0.3001
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 160ms/step - acc: 0.8830 - loss: 0.2878
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 168ms/step - acc: 0.8931 - loss: 0.2644
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 161ms/step - acc: 0.9002 - loss: 0.2429
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 147ms/step - acc: 0.9135 - loss: 0.2171
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 162ms/step - acc: 0.9177 - loss: 0.2075

 총 학습 시간 : 226.65초


In [ ]:
# 6번 블록

# RNN 평가

score = model.evaluate(x=X_test,
                       y=Y_test,
                       verbose = 1)

print('최종 정확도: {:.2f}'.format(score[1]))

# RNN 예측
test = X_test[MY_SAMPLE].reshape(1, 80)
pred = model.predict(test)
pred = (pred > 0.5)

# 감성 예측 출력
print('\n샘플 영화평:\n', test)
print('RNN 감성 예측 : ',pred)
print('정답 (0=부정, 1=긍정) : ', Y_test[MY_SAMPLE])

782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - acc: 0.7813 - loss: 0.6308
최종 정확도: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step

샘플 영화평:
 [[   1  146  427    2   14   20  218  112 2962   32   37  119   14   20
   144    2  910    5    2    4 4659   18   12 3403  853   28    8 2225
    12   95  474  818 4651   18 1462   13  124  285    5 1462   11   14
    20  122    6   52  292    5   13  774 2626   46  138  910 1481  276
    14   20   23  288   42   23 1856   11 2364    2   33  222   13  774
   110  101 4651   14    9    6 3799   52   20    5]]
RNN 감성 예측 :  [[ True]]
정답 (0=부정, 1=긍정) :  1
